In [ ]:
import Trader
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

#considering the fees are different depending on the service used, fees are not included on this code. I intend to use it on Ligther or any other perp that has zero fees.

In [ ]:
def get_data(symbol, timeframe, year_span= 1):                                                                          #define function to calculate dates and get data
    end_date= datetime.now()
    start_date= end_date - timedelta(days= 365 * year_span)                                                             #calculates the start date
    return Trader.Trader().get_ohlc_range(symbol, timeframe, start_date, end_date)                                      #uses function from Trader

In [ ]:
def calculate_macd(df, light_ema= 12, heavy_ema= 26, signal_ema= 9):                                                    #function to calculate macd, signal and histogram
    df['ema_short']= df['close'].ewm(span= light_ema, adjust= False).mean()
    df['ema_long']= df['close'].ewm(span= heavy_ema, adjust= False).mean()
    df['macd']= df['ema_short'] - df['ema_long']
    df['signal']= df['macd'].ewm(span= signal_ema, adjust= False).mean()
    df['histogram']= df['macd'] - df['signal']
    df['bullish cross']= (df['macd'] > df['signal']) & (df['macd'].shift(1) <= df['signal'].shift(1))                   #triggers for the trade
    df['bearish cross']= (df['macd'] < df['signal']) & (df['macd'].shift(1) >= df['signal'].shift(1))

    return df

In [ ]:
def trade_macd(df):                                                                                                     #function to test the strategy based on previous data
    open_position= 0
    entry_price= None
    exit_price= None
    trades= []

    dict_map= {j:i for i, j in enumerate(df.columns)}
    vals= df.values

    for k in range(1, len(vals)):
        prev= vals[k-1]                                                                                                  
        curr= vals[k]

        if open_position== 0 and prev[dict_map['bullish cross']]== True:                                                #considering we only do trades after a candle has closed, we open a trade on the previous data
            open_position= 1
            entry_price= curr[dict_map['close']]
            trades+= [{
            'time': curr[dict_map['time']],
            'price': curr[dict_map['close']],
            'macd': curr[dict_map['macd']],
            'signal': curr[dict_map['signal']],
            'side': 'long'
        }]

        elif open_position== 1 and prev[dict_map['bearish cross']]== True:
            open_position= 0
            exit_price= curr[dict_map['close']]
            trades+= [{
            'time': curr[dict_map['time']],
            'price': curr[dict_map['close']],
            'macd': curr[dict_map['macd']],
            'signal': curr[dict_map['signal']],
            'side': 'short',
            'return %': (exit_price/entry_price) -1                                                                     #return in of a trade in percentile
        }]
    
    df_trades= pd.DataFrame(trades)
    df_trades.set_index('time', inplace= True)
    return df_trades



In [ ]:
def test_macd(symbol, timeframe, year_span= 1, light_ema= 12, heavy_ema= 26, signal_ema= 9, fig= None):
    df_data= get_data(symbol, timeframe, year_span)
    df_data= calculate_macd(df_data, light_ema, heavy_ema, signal_ema)
    df_trades= trade_macd(df_data)
    roi = df_trades['return %'].sum()                                                                                   #return over investiment of the strategy
    return df_trades, df_data, roi, light_ema, heavy_ema, signal_ema

In [ ]:
#positive_return = 0
#negative_return = 0
#neutral_return = 0
#cutoff = 50

 
symbol= 'ETHUSD'
tf= 'M30' 
timeframes= ['M20', 'M30']
h_ema=[50, 200] 
l_ema=[12, 26]
s_ema=[9]

for tf in timeframes:
    for heavy_ema in h_ema:
        for light_ema in l_ema:
            for signal_ema in s_ema:
                df_trades, df_data, roi, light_ema, heavy_ema, signal_ema= test_macd(symbol, tf, heavy_ema= heavy_ema, light_ema= light_ema, signal_ema= signal_ema)
                df_trades['return %'].dropna().cumsum().plot(label=f'Strategy (tf= {tf}, H_EMA= {heavy_ema} , L_EMA= {light_ema} , S_EMA= {signal_ema})')
                print(f'Strategy (tf= {tf}, H_EMA= {heavy_ema} , L_EMA= {light_ema} , S_EMA= {signal_ema} ) had a return of {df_trades["return %"].sum()*100:.2f}%')

plt.legend(loc='center left', bbox_to_anchor=(1,0.5))  
plt.show()    


In [ ]:
_, df_data, *_= test_macd('ETHUSD', 'M30', year_span= (1/12))

plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(df_data.index, df_data['close'], label= 'Close Price')
plt.scatter(df_data[df_data['bullish cross']== True].index, df_data[df_data['bullish cross']== True]['close'], label= 'Bullish Cross', marker= '^', color='green', zorder= 2, s=125)
plt.scatter(df_data[df_data['bearish cross']== True].index, df_data[df_data['bearish cross']== True]['close'], label= 'Bearish Cross', marker= 'v', color= 'red', zorder= 2, s=125)
plt.title(f'{symbol} Close Price')
plt.ylabel('Price')
plt.legend()
plt.xlim(df_data.index.min(), df_data.index.max())


plt.subplot(2, 1, 2)
plt.plot(df_data.index, df_data['macd'], label= 'MACD')
plt.plot(df_data.index, df_data['signal'], label= 'signal')
plt.scatter(df_data[df_data['bullish cross']== True].index, df_data[df_data['bullish cross']== True]['macd'], label= 'Bullish Cross', marker= '^', color='green', zorder= 2, s=125)
plt.scatter(df_data[df_data['bearish cross']== True].index, df_data[df_data['bearish cross']== True]['macd'], label= 'Bearish Cross', marker= 'v', color= 'red', zorder= 2, s=125)
plt.bar(df_data.index, df_data['histogram'], label= 'Histogram', width= 0.05, color= ['green' if y >= 0 else 'red' for y in df_data['histogram']], alpha= 0.7, edgecolor= 'black', linewidth= 0.6)
plt.title('MACD and Signal Line')
plt.ylabel('MACD/Signal')
plt.legend()
plt.xlim(df_data.index.min(), df_data.index.max())
plt.ylim(df_data['macd'].min() - 1, df_data['macd'].max() + 1)


plt.tight_layout()
plt.show()